In [ ]:
dataset_name = 'cardiovascular_disease'
target_column_name = 'cardio' 

# Model comparison
## Load requirements
### Load Dataset

In [ ]:
import pandas as pd

clean_df = pd.read_csv('Data/'+dataset_name+'/clean_dataset.csv')

all = clean_df.columns.drop(target_column_name)
data_df = clean_df[all]
target_df = clean_df[target_column_name]

### Load optimized model parameters

In [ ]:
import json

param_file = open('Data/'+dataset_name+'/tuned_hyperparameters.json')
models = json.load(param_file)
param_file.close()

### Import all models

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

## Cross-Validation evaluation

In [ ]:
from sklearn.model_selection import cross_val_score
from statistics import mean
import warnings

n_cv_folds = 5
models_scores = []
for model in models :
    model_class =  globals()[model['model_class_name']]
    model['model'] = model_class(**model['best_params'])
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        cv_scores = cross_val_score(model['model'], data_df, target_df, cv=n_cv_folds)
    model_scores = {'Model': model['name']}
    for i, score in enumerate(cv_scores):
        model_scores['Fold '+str(i+1)] = score 
    model_scores['Average'] = mean(cv_scores)
    models_scores.append(model_scores)

scores_df = pd.DataFrame.from_dict(models_scores)
display(scores_df)

## Trace RoC curves

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

x_train, x_test, y_train, y_test = train_test_split(data_df, target_df, test_size=0.2)

if set(y_test.unique()) == set([0, -1]): 
    y_test = y_test.map({0: 1, -1: 0}).astype(int)
for model in models :
    model_class =  globals()[model['model_class_name']]
    model['model'] = model_class(**model['best_params'])
    model['model'].fit(x_train,y_train)

    y_scores = model['model'].predict_proba(x_test)

    fpr, tpr, threshold = roc_curve(y_test, y_scores[:, 1])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=model['name']+' (%0.2f AUC)' % roc_auc)


plt.title('RoC curves per model')
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()